In [1]:
%matplotlib notebook
import torch
import matplotlib.pyplot as plt
from lazy_imports import np
from lazy_imports import plt
from lazy_imports import sitk
from lazy_imports import LinearOperator, gmres
from lazy_imports import loadmat, savemat
from lazy_imports import sitk

from lazy_imports import itkwidgets
from lazy_imports import itkview
from lazy_imports import interactive
from lazy_imports import ipywidgets
from lazy_imports import pv
from IPython.core.display import display, HTML
plt.rcParams["figure.figsize"] = (4, 4) # (w, h)
%matplotlib notebook
display(HTML("<style>.container { width:55% !important; }</style>"))

In [2]:
from disp.vis import vis_tensors, vis_path, disp_scalar_to_file
from disp.vis import disp_vector_to_file, disp_tensor_to_file
from disp.vis import disp_gradG_to_file, disp_gradA_to_file
from disp.vis import view_3d_tensors, tensors_to_mesh

In [3]:
from data.io import readRaw, ReadTensors, ReadScalars, WriteTensorNPArray
from data.convert import GetNPArrayFromSITK, GetSITKImageFromNP
import data.gen

In [4]:
from util import diff
from util import tensors
from util import maskops as mo
from util.riemann import riem_vec_norm
from util.tensors import scale_by_alpha

In [5]:
from mtch.RegistrationFunc3D import *
from mtch.SplitEbinMetric3D import *
from mtch.GeoPlot import *

In [6]:
import algo.metricModSolver2d as mms
import algo.geodesic as geo
import algo.euler as euler
import algo.dijkstra as dijkstra

In [7]:
input_dir = '/usr/sci/projects/HCP/Kris/NSFCRCNS/TestResults/working_3d_python'
output_dir = 'Cubic12Geo'

In [8]:
index0, index1 = 1, 2
g00 = torch.from_numpy(sitk.GetArrayFromImage(sitk.ReadImage(f'{input_dir}/cubic{index0}_scaled_tensors.nhdr'))).double().permute(3,2,1,0)
g11 = torch.from_numpy(sitk.GetArrayFromImage(sitk.ReadImage(f'{input_dir}/cubic{index1}_scaled_tensors.nhdr'))).double().permute(3,2,1,0)

height, width, depth = 100,100,41
g0, g1 = torch.zeros(height, width, depth, 3, 3, dtype=torch.double), torch.zeros(height, width, depth, 3, 3, dtype=torch.double)

g0[:,:,:,0,0] = g00[0]
g0[:,:,:,0,1] = g00[1]
g0[:,:,:,0,2] = g00[2]
g0[:,:,:,1,0] = g00[1]
g0[:,:,:,1,1] = g00[3]
g0[:,:,:,1,2] = g00[4]
g0[:,:,:,2,0] = g00[2]
g0[:,:,:,2,1] = g00[4]
g0[:,:,:,2,2] = g00[5]
g1[:,:,:,0,0] = g11[0]
g1[:,:,:,0,1] = g11[1]
g1[:,:,:,0,2] = g11[2]
g1[:,:,:,1,0] = g11[1]
g1[:,:,:,1,1] = g11[3]
g1[:,:,:,1,2] = g11[4]
g1[:,:,:,2,0] = g11[2]
g1[:,:,:,2,1] = g11[4]
g1[:,:,:,2,2] = g11[5]

In [9]:
Tpts = 7
geo_set = get_geo(g0, g1, 1./3., Tpts)

This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/torch/csrc/utils/python_arg_parser.cpp:882.)


In [10]:
for s in range(Tpts):
    atlas_lin = np.zeros((6,height,width,depth))
    atlas_lin[0] = geo_set[s][:,:,:,0,0]
    atlas_lin[1] = geo_set[s][:,:,:,0,1]
    atlas_lin[2] = geo_set[s][:,:,:,0,2]
    atlas_lin[3] = geo_set[s][:,:,:,1,1]
    atlas_lin[4] = geo_set[s][:,:,:,1,2]
    atlas_lin[5] = geo_set[s][:,:,:,2,2]
    sitk.WriteImage(sitk.GetImageFromArray(np.transpose(atlas_lin,(3,2,1,0))), f'{output_dir}/cubic12_{s}_tens.nhdr')
    sitk.WriteImage(sitk.GetImageFromArray(np.transpose(torch.where(torch.det(geo_set[s])==1,0,1).numpy(),(2,1,0))), f'{output_dir}/cubic12_{s}_mask.nhdr')

RuntimeError: Exception thrown in SimpleITK ImageFileWriter_Execute: /tmp/SimpleITK-build/ITK/Modules/IO/NRRD/src/itkNrrdImageIO.cxx:1116:
itk::ERROR: itk::ERROR: NrrdImageIO(0x55f95ae562d0): Write: Error writing Cubic12Geo/cubic12_0_tens.nhdr:
[nrrd] nrrdSave: couldn't fopen("Cubic12Geo/cubic12_0_tens.nhdr","wb"): No such file or directory


In [ ]:
geo_delta_t = 0.1
geo_iters = 1300 
start_coords = [[13, 14, 20]] # golden test start point
init_velocities = [None]
path_set = []

s = 0
tensor_lin = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_tens.nhdr')),(3,2,1,0))
mask = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_mask.nhdr')),(2,1,0))
geox, geoy, geoz = geo.geodesicpath_3d(tensor_lin, mask,\
                                            start_coords[0], init_velocities[0], \
                                            geo_delta_t, iter_num=geo_iters, both_directions=False)
path_set.append((geox[:-1],geoy[:-1],geoz[:-1]))
vwr=view_3d_tensors(np.transpose(tensor_lin,(1,2,3,0)),mask,tensor_lin[3,:,:,:],paths=path_set,stride=6,scale=6)
vwr

In [ ]:
s = 1
path_set = []
tensor_lin = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_tens.nhdr')),(3,2,1,0))
mask = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_mask.nhdr')),(2,1,0))
geox, geoy, geoz = geo.geodesicpath_3d(tensor_lin, mask,\
                                            start_coords[0], init_velocities[0], \
                                            geo_delta_t, iter_num=geo_iters, both_directions=False)
path_set.append((geox[:-1],geoy[:-1],geoz[:-1]))
vwr=view_3d_tensors(np.transpose(tensor_lin,(1,2,3,0)),mask,tensor_lin[3,:,:,:],paths=path_set,stride=6,scale=6)
vwr

In [42]:
s = 2
path_set = []
tensor_lin = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_tens.nhdr')),(3,2,1,0))
mask = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_mask.nhdr')),(2,1,0))
geox, geoy, geoz = geo.geodesicpath_3d(tensor_lin, mask,\
                                            start_coords[0], init_velocities[0], \
                                            geo_delta_t, iter_num=geo_iters, both_directions=False)
path_set.append((geox[:-1],geoy[:-1],geoz[:-1]))
vwr=view_3d_tensors(np.transpose(tensor_lin,(1,2,3,0)),mask,tensor_lin[3,:,:,:],paths=path_set,stride=6,scale=6)
vwr

Finding geodesic path from [13, 14, 20] with initial velocity [0.30259403 0.95311954 0.        ]
Found 0 voxels where unable to take 1st derivative.
Found 0 reduced accuracy 2nd derivative voxels.
numpts 135
smallest,largest max eigenvalue 1.5801098277369667 9.559687055469483


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [41]:
s = 3
path_set = []
tensor_lin = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_tens.nhdr')),(3,2,1,0))
mask = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_mask.nhdr')),(2,1,0))
geox, geoy, geoz = geo.geodesicpath_3d(tensor_lin, mask,\
                                            start_coords[0], init_velocities[0], \
                                            geo_delta_t, iter_num=geo_iters, both_directions=False)
path_set.append((geox[:-1],geoy[:-1],geoz[:-1]))
vwr=view_3d_tensors(np.transpose(tensor_lin,(1,2,3,0)),mask,tensor_lin[3,:,:,:],paths=path_set,stride=6,scale=6)
vwr

Finding geodesic path from [13, 14, 20] with initial velocity [0.28788748 0.95766424 0.        ]
Found 0 voxels where unable to take 1st derivative.
Found 0 reduced accuracy 2nd derivative voxels.
numpts 135
smallest,largest max eigenvalue 1.8475599159179434 8.83122635409812


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
s = 4
tensor_lin = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_tens.nhdr')),(3,2,1,0))
mask = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_mask.nhdr')),(2,1,0))
geox, geoy, geoz = geo.geodesicpath_3d(tensor_lin, mask,\
                                            start_coords[0], init_velocities[0], \
                                            geo_delta_t, iter_num=geo_iters, both_directions=False)
path_set.append((geox[:-1],geoy[:-1],geoz[:-1]))
vwr=view_3d_tensors(np.transpose(tensor_lin,(1,2,3,0)),mask,tensor_lin[3,:,:,:],paths=path_set,stride=6,scale=6)
vwr

In [ ]:
s = 5
tensor_lin = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_tens.nhdr')),(3,2,1,0))
mask = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_mask.nhdr')),(2,1,0))
geox, geoy, geoz = geo.geodesicpath_3d(tensor_lin, mask,\
                                            start_coords[0], init_velocities[0], \
                                            geo_delta_t, iter_num=geo_iters, both_directions=False)
path_set.append((geox[:-1],geoy[:-1],geoz[:-1]))
vwr=view_3d_tensors(np.transpose(tensor_lin,(1,2,3,0)),mask,tensor_lin[3,:,:,:],paths=path_set,stride=6,scale=6)
vwr

In [ ]:
s = 6
tensor_lin = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_tens.nhdr')),(3,2,1,0))
mask = np.transpose(sitk.GetArrayFromImage(sitk.ReadImage(f'{output_dir}/cubic12_{s}_mask.nhdr')),(2,1,0))
geox, geoy, geoz = geo.geodesicpath_3d(tensor_lin, mask,\
                                            start_coords[0], init_velocities[0], \
                                            geo_delta_t, iter_num=geo_iters, both_directions=False)
path_set.append((geox[:-1],geoy[:-1],geoz[:-1]))
vwr=view_3d_tensors(np.transpose(tensor_lin,(1,2,3,0)),mask,tensor_lin[3,:,:,:],paths=path_set,stride=6,scale=6)
vwr